Get packages

In [2]:
import os

os.chdir(os.path.join(os.getcwd(), '..'))
import src.preprocessing
import src.model_training

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time
import shap
import lightgbm as lgb

import json
import sys
import pickle

from verstack import LGBMTuner
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Get data & model

In [22]:
df = src.preprocessing.get_train_data('data/data_pricing_challenge.csv')

In [5]:
pkl_file_path  = 'models/model_baseline'
# Load the model
with open(pkl_file_path, 'rb') as f:
    model = pickle.load(f)

Prepare data for prediction

In [29]:
variables = src.model_training.txt_load('utils/model_variables.txt')
df = df.loc[df['price'] >= 20000]
df['mileage'] = df['mileage']+10000
df['antiquity'] = df['antiquity']+1
df_pred = df[variables]
df.shape, df_pred.shape

((999, 39), (999, 19))

In [26]:
# Make predictions
prediction = model.predict(df_pred)

# Add prediction to dataset
df_pred['price_pred'] = prediction

In [27]:
df_pred.head()

,mileage,engine_power,antiquity,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,convertible,coupe,estate,hatchback,sedan,subcompact,suv,van,price_pred
1,23929,317,2.995893,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,47342.957600
3,138035,135,4.986311,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,19828.177062
4,107097,160,4.978097,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,21832.796684
12,142025,135,5.000000,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,16734.590501
13,87061,135,3.986995,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,25091.034822


In [28]:
df.head()

,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at,antiquity,diesel,electro,hybrid_petrol,petrol,beige,black,blue,brown,green,grey,orange,red,silver,white,convertible,coupe,estate,hatchback,sedan,subcompact,suv,van
0,140411,100,2012-01-02,diesel,black,convertible,1,1,0,0,1,1,1,0,11300,2018-01-01,5.998631,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,13929,317,2016-01-04,petrol,grey,convertible,1,1,0,0,0,1,1,1,69700,2018-01-02,1.995893,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,183297,120,2012-01-04,diesel,white,convertible,0,0,0,0,1,0,1,0,10200,2018-01-02,5.995893,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
3,128035,135,2014-01-07,diesel,red,convertible,1,1,0,0,1,1,1,1,25100,2018-01-02,3.986311,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
4,97097,160,2014-01-12,diesel,silver,convertible,1,1,0,0,0,1,1,1,33400,2018-01-04,3.978097,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
